# VariantFormer: Variant-to-Expression Prediction

## Predicting the Functional Impact of Genetic Variants on Gene Expression

### Executive Summary

VariantFormer is a state-of-the-art deep learning framework that predicts how genetic variants affect gene expression across diverse tissues and populations. This notebook demonstrates the core functionality of the VariantFormer system, showcasing how single nucleotide variants (SNVs) can be analyzed to understand their potential impact on cellular function.

### Innovation

VariantFormer addresses several critical limitations of existing approaches:

1. **Multi-tissue Analysis**: Predicts variant effects across 64+ tissues and cell types
2. **Population-aware**: Considers genetic background from major world populations
3. **Mechanistic Understanding**: Models both cis-regulatory elements (CREs) and gene sequences
---


## Methodology Overview

VariantFormer employs a sophisticated **transformer-based architecture** that integrates DNA sequences around genes and regulatory elements.

### Scoring Strategies

#### 1. Expression Prediction Scores
- **Direct Output**: Predicted log-transformed gene expression values
- **Comparison**: Reference (wild-type) vs. heterozygous vs. homozygous variant genotypes
- **Tissue-specific**: Separate predictions for each tissue type

#### 2. Log2 Fold Change (log2fc) Regulation Scores as eQTLs
- **Population Comparison**: Analyzes expression differences across populations
- **Reference Comparison**: Compares variant expression to reference genome expression  
- **Directional Interpretation**: 
  - **Positive log2fc (>0)**: Gene upregulation - variant increases expression
  - **Negative log2fc (<0)**: Gene downregulation - variant decreases expression  
  - **log2fc ≈ 0**: Minimal effect - variant has little impact
- **Population Weighting**: Aggregates scores across populations using allele frequency weights
- **Magnitude**: Larger absolute values indicate stronger regulatory effects

In [2]:
# Essential imports
import sys
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

sys.path.append(str(Path.cwd().parent))

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from processors.variantprocessor import VariantProcessor


Using hardcoded manifest path for GeneSequencesManifestLookup
Using hardcoded manifest path for CreSequencesManifestLookup


In [3]:
# Initialize VariantFormer
print("🚀 Initializing VariantFormer Variant Processor...")
model_class = 'D2C_AG' # model class can be 'D2C_AG', 'D2C_PCG'. AG model is all-genes model trained on both protein-coding and non-coding genes.
# model_class = 'D2C_PCG' # Uncomment to use the PCG model
vep = VariantProcessor(model_class=model_class)

print("✅ System initialized!")
print(f"🌍 Populations: {', '.join(vep.populations)}")
print(f"🧬 Tissues: {len(vep.tissue_vocab)} available")

2025-10-31 18:26:36 - utils.assets - WARNING - Using hardcoded manifest path for GeneSequencesManifestLookup
2025-10-31 18:26:36 - utils.assets - WARNING - Using hardcoded manifest path for CreSequencesManifestLookup


🚀 Initializing VariantFormer Variant Processor...
✅ System initialized!
🌍 Populations: REF_HG38, EAS, EUR, AFR, SAS, AMR
🧬 Tissues: 62 available


In [4]:
# Define variants for analysis
print("🧬 Creating multi-variant dataset...")

multi_variants = {
    "chr": ["chr13"],
    "pos": [
        113978728,
    ],
    "ref": [
        "A",
    ],
    "alt": ["G"],
    "tissue": ["whole blood"],
    "gene_id": ["ENSG00000185989.10"],
}

variant_df = pd.DataFrame(multi_variants)
print(variant_df.to_string(index=False))


🧬 Creating multi-variant dataset...
  chr       pos ref alt      tissue            gene_id
chr13 113978728   A   G whole blood ENSG00000185989.10


In [5]:
# Run variant predictions
print("🔬 Running VariantFormer variant analysis...")
output_dir = "/tmp/vep_multi_output"
# Predict expression effects for all variants
raw_predictions = vep.predict(var_df=variant_df, output_dir=output_dir)

🔬 Running VariantFormer variant analysis...
Initializing Variant Processor...
Loading gene annotations...
Loading CRE annotations...
Loading variants...
Loaded 1 variants
Loaded 1 variants for processing
Mapped 1 gene-variant pairs
Loading BPE encoder...
Loaded BPE vocabulary from /work/vocabs/bpe_vocabulary_500_using_huggingface.json
Loading model...
Loading Seq2Reg model...
Loading Seq2Reg gene model...
Creating Seq2Gene model...
Model class: <class 'seq2gene.model_combined_modulator.Seq2GenePredictorCombinedModulator'>
Model architecture:
Model: Seq2GenePredictorCombinedModulator
  start_tkn: 96,768 params
  cre_tokenizer: 31,826,153 params
  gene_tokenizer: 31,826,153 params
  gene_map: 787,968 params
  cre_map: 787,968 params
  combined_modulator: 1,157,298,176 params
  tissue_heads: 4,726,273 params
  gene_loss: 0 params
Total number of parameters: 1,227,349,459
Loading checkpoint from /work/_artifacts/v4_ag_epoch9_checkpoint.pth


Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Model loaded successfully on cuda
Initializing trainer...
Initialization complete!


Predicting: |          | 0/? [00:00<?, ?it/s]

2025-10-31 18:26:53 - utils.assets - INFO - Loading parquet file: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/youssef/src/variantformer/_artifacts/cres_seqs_manifest.parquet
2025-10-31 18:26:53 - utils.assets - INFO - Loading parquet file: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/youssef/src/variantformer/_artifacts/cres_seqs_manifest.parquet
2025-10-31 18:26:53 - utils.assets - INFO - Loading parquet file: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/youssef/src/variantformer/_artifacts/cres_seqs_manifest.parquet
2025-10-31 18:26:53 - utils.assets - INFO - Loading parquet file: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/youssef/src/variantformer/_artifacts/cres_seqs_manifest.parquet
2025-10-31 18:26:53 - utils.assets - INFO - Loading parquet file: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/youssef/src/variantformer/_artifacts/cres_seqs_manifest.parquet
2025-10-31 18:26:53 - utils.assets - INFO - Validated schema - found columns: {'file_path', 'chromosome', '__index_level_0__', 'po

Predictions saved to /tmp/vep_multi_output/vep_D2C.parquet
Cleanup complete


In [ ]:
# Explain prediction output
print("🧬 Explaining prediction output..."
      )
print(f"Columns in the prediction output dataframe:{', '.join(raw_predictions.columns)}")
print('chrom: Chromosome of the variant')
print('pos: Position of the variant')
print('ref: Reference allele')
print('alt: Alternate allele')
print('gene_id: Ensembl Gene ID')
print('tissue: Tissue name')
print('population: Population code')
print('sample_name: 1000 genome sample identifier')
print('zygosity: Zygosity of the variant in the sample (0: homozygous ref, 1: heterozygous, 2: homozygous alt) when the background genome is \{population\}')
print('gene_exp: Predicted gene expression level for the gene in the specified tissue and sample')
print('gene_token_embedding: Embedding of the token that overlaps with the variant position from the last transformer layer')
print('cre_token_embeddings: Embeddings of regulatory elements overlapping with the variant position from the last transformer layer')
print('gene_emb: Embedding of the gene conditioned on tissue and regulatory context from the last transformer layer')
print('variant_type: Gene Overlap/CRE Overlap/Intergenic variant type annotation')

🧬 Explaining prediction output...
Columns in the prediction output dataframe:chrom, pos, ref, alt, genes, tissues, variant_type, population, sample_name, zygosity, gene_exp, gene_emb, gene_token_embedding, cre_token_embedding
chrom: Chromosome of the variant
pos: Position of the variant
ref: Reference allele
alt: Alternate allele
gene_id: Ensembl Gene ID
tissue: Tissue name
population: Population code
sample_name: 1000 genome sample identifier
zygosity: Zygosity of the variant in the sample (0: homozygous ref, 1: heterozygous, 2: homozygous alt) when the background genome is \{population\}
gene_exp: Predicted gene expression level for the gene in the specified tissue and sample
gene_token_embedding: Embedding of the token that overlaps with the variant position from the last transformer layer
cre_token_embeddings: Embeddings of regulatory elements overlapping with the variant position from the last transformer layer
gene_emb: Embedding of the gene conditioned on tissue and regulatory c

In [11]:
raw_predictions.head()

,chrom,pos,ref,alt,genes,tissues,variant_type,population,sample_name,zygosity,gene_exp,gene_emb,gene_token_embedding,cre_token_embedding
0,chr13,113978728,A,G,ENSG00000185989.10,whole blood,Gene overlap only,REF_HG38,hg38,2,3.182971,"[4.90625, 0.16699219, -0.5546875, -2.546875, 6...","[0.87109375, -0.8671875, -1.0078125, -0.578125...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,chr13,113978728,A,G,ENSG00000185989.10,whole blood,Gene overlap only,REF_HG38,hg38,1,3.182971,"[4.875, 0.19042969, -0.5234375, -2.5625, 6.281...","[0.80859375, -0.67578125, -0.45703125, 0.10937...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,chr13,113978728,A,G,ENSG00000185989.10,whole blood,Gene overlap only,REF_HG38,hg38,0,3.182971,"[4.90625, 0.18164062, -0.5390625, -2.5625, 6.2...","[0.96484375, -0.875, -0.7265625, -0.3671875, 2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,chr13,113978728,A,G,ENSG00000185989.10,whole blood,Gene overlap only,EAS,HG00404,2,3.454005,"[4.625, 0.19140625, 2.515625, 0.12695312, 5.37...","[2.9375, -1.4765625, -0.328125, -1.6875, 2.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,chr13,113978728,A,G,ENSG00000185989.10,whole blood,Gene overlap only,EAS,HG00404,1,3.454005,"[4.625, 0.19140625, 2.515625, 0.12695312, 5.37...","[2.9375, -1.4765625, -0.328125, -1.6875, 2.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


### Step 2: Format Prediction Scores

This step formats the raw predictions into a structured format for downstream analysis. The formatting process includes:

1. **Data Transformation**: Restructure predictions to compare expression across populations
2. **Reference Comparison**: Use HG38 reference genome as baseline
3. **Population Comparison**: Organize expression differences across major world populations
4. **Data Pivoting**: Transform data into population-tissue matrix format for analysis


In [14]:
# Format and score the predictions
print("📊 Formatting and scoring predictions...")

# Step 1: Format scores
formatted_scores = vep.format_scores(raw_predictions)


# Step 2: Calculate eQTL scores (log2fc-based)
final_results = vep.eqtl_scores(formatted_scores)
print(f"✅ Calculated regulation scores for {len(final_results)} variants")
final_results.head()

📊 Formatting and scoring predictions...
   REF_HG38-0-exp  AFR-2-exp  AMR-2-exp  EAS-2-exp  EUR-2-exp  REF_HG38-2-exp  \
0        3.182971   3.303087   3.333206   3.454005   3.242956        3.182971   

   SAS-2-exp           variant_id               genes      tissues  ...  \
0   3.273004  chr13_113978728_A_G  ENSG00000185989.10  whole blood  ...   

  D2C-EAS-2-exp-log2fc D2C-EUR-2-exp-log2fc D2C-REF_HG38-2-exp-log2fc  \
0             0.117896             0.026935                       0.0   

   D2C-SAS-2-exp-log2fc        AF    AF_EUR    AF_EAS    AF_AMR    AF_SAS  \
0              0.040241  0.282324  0.368088  0.354701  0.354082  0.327787   

     AF_AFR  
0  0.104143  

[1 rows x 26 columns]
✅ Calculated regulation scores for 1 variants


,variant_id,genes,tissues,ref,alt,chr,pos,D2C-agg-log2fc-weighted,D2C-AFR-2-exp-log2fc,D2C-AMR-2-exp-log2fc,D2C-EAS-2-exp-log2fc,D2C-EUR-2-exp-log2fc,D2C-REF_HG38-2-exp-log2fc,D2C-SAS-2-exp-log2fc
0,chr13_113978728_A_G,ENSG00000185989.10,whole blood,A,G,chr13,113978728,0.062333,0.053441,0.066536,0.117896,0.026935,0.0,0.040241


In [15]:
print('D2C-AFR-2-exp-log2fc: Log2 fold change in expression for the homozygous alternative allele in the AFR background genome compared to reference')
print('D2C-EUR-2-exp-log2fc: Log2 fold change in expression for the homozygous alternative allele in the EUR background genome compared to reference')
print('D2C-EAS-2-exp-log2fc: Log2 fold change in expression for the homozygous alternative allele in the EAS background genome compared to reference')
print('D2C-SAS-2-exp-log2fc: Log2 fold change in expression for the homozygous alternative allele in the SAS background genome compared to reference')
print('D2C-AMR-2-exp-log2fc: Log2 fold change in expression for the homozygous alternative allele in the AMR background genome compared to reference')
print('D2C-AMR-2-exp-log2fc: Log2 fold change in expression for the homozygous alternative allele in the AMR background genome compared to reference')
print('D2C-agg-log2fc-weighted: Weighted aggregate log2 fold change in expression across all populations based on their allele frequencies')
print('D2C-REF_HG38-2-exp-log2fc: Log2 fold change in expression for the homozygous alternative allele in the HG38 reference genome compared to reference homozygous ref')


D2C-AFR-2-exp-log2fc: Log2 fold change in expression for the homozygous alternative allele in the AFR background genome compared to reference
D2C-EUR-2-exp-log2fc: Log2 fold change in expression for the homozygous alternative allele in the EUR background genome compared to reference
D2C-EAS-2-exp-log2fc: Log2 fold change in expression for the homozygous alternative allele in the EAS background genome compared to reference
D2C-SAS-2-exp-log2fc: Log2 fold change in expression for the homozygous alternative allele in the SAS background genome compared to reference
D2C-AMR-2-exp-log2fc: Log2 fold change in expression for the homozygous alternative allele in the AMR background genome compared to reference
D2C-AMR-2-exp-log2fc: Log2 fold change in expression for the homozygous alternative allele in the AMR background genome compared to reference
D2C-agg-log2fc-weighted: Weighted aggregate log2 fold change in expression across all populations based on their allele frequencies
D2C-REF_HG38-2-e

**Score Interpretation:**
- **log2fc > 0**: Upregulation (variant increases gene expression)
  - log2fc ≥ 1.0: Strong upregulation (≥2-fold increase)
  - log2fc ≥ 0.5: Moderate upregulation (≥1.4-fold increase)
- **log2fc ≈ 0**: Neutral effect (variant has minimal impact)
- **log2fc < 0**: Downregulation (variant decreases gene expression)
  - log2fc ≤ -0.5: Moderate downregulation (≤0.7-fold decrease)
  - log2fc ≤ -1.0: Strong downregulation (≤0.5-fold decrease)


## Key Findings & Applications

### Variant-level Insights
VariantFormer enables simultaneous analysis of variant sets to:
- **Prioritize variants** by regulatory impact
- **Compare effects** across different genomic loci  
- **Identify tissue-specific** regulatory variants
- **Population-stratified** variant interpretation

### Clinical Applications
🏥 **Variant Prioritization**: Focus on functionally relevant variants  
🧬 **Personalized Medicine**: Population-aware variant interpretation  
🔬 **Drug Development**: Identify expression-modulating variants  
📊 **GWAS Follow-up**: Prioritize causal variants from association studies  

---

## Summary

VariantFormer provides a **scalable, population-aware framework** for predicting variant effects on gene expression. The multi-variant capability enables efficient analysis of variant sets while maintaining tissue-specific and population-stratified insights.

**Key Advantages:**
- Multi-variant batch processing
- 64+ tissue types supported  
- 6 major world populations


# The following example demonstrates how VariantFormer can be used as insilico tool with personal genomic data:


In [16]:
# Example: VCF-based Analysis
print("🧬 VCF Integration Example")
print("=" * 40)

# VCF files contain individual genotype information
# This enables more precise analysis based on actual sample data

🧬 VCF Integration Example


In [17]:
# For VCF analysis, we can reuse the existing VariantProcessor
print("🔄 Using existing VariantFormer system for VCF analysis...")
print("📁 VCF files enable individual-specific genomic analysis")

🔄 Using existing VariantFormer system for VCF analysis...
📁 VCF files enable individual-specific genomic analysis


In [18]:
model_class = 'D2C_AG' # model class can be 'D2C_AG', 'D2C_PCG'. AG model is all-genes model trained on both protein-coding and non-coding genes.
# model_class = 'D2C_PCG' # Uncomment to use the PCG model
vep = VariantProcessor(model_class=model_class)

2025-10-31 18:54:37 - utils.assets - WARNING - Using hardcoded manifest path for GeneSequencesManifestLookup
2025-10-31 18:54:37 - utils.assets - WARNING - Using hardcoded manifest path for CreSequencesManifestLookup


In [19]:
# Example VCF integration parameters
vcf_path = os.path.join(str(Path.cwd().parent),"_artifacts/HG00096.vcf.gz") # Change path as needed
sample_name = "HG00096"  # European sample from 1000 Genomes Project


# Same variant for insilico analysis. Alt allele will be placed in the sample genotype context at that specific position. 
# Ref is from reference genome.
vcf_variant_data = {
    "chr": ["chr13"],
    "pos": [113978728],
    "ref": ["A"],
    "alt": ["G"],
    "tissue": ["whole blood"],
    "gene_id": ["ENSG00000185989.10"],
}

vcf_variant_df = pd.DataFrame(vcf_variant_data)
print("\n🧬 Analyzing the same variant with individual genotype data:")
print(vcf_variant_df.to_string(index=False))


🧬 Analyzing the same variant with individual genotype data:
  chr       pos ref alt      tissue            gene_id
chr13 113978728   A   G whole blood ENSG00000185989.10


In [20]:
# Run VariantFormer with VCF integration
print("\n🔬 Running VariantFormer analysis with VCF integration...")
print("⏳ Processing individual genotype data...")

vcf_predictions = vep.predict(
    var_df=vcf_variant_df,
    output_dir="/tmp/vep_output_vcf",
    vcf_path=vcf_path,
    sample_name=sample_name,
)

print("✅ VCF-based predictions completed!")


🔬 Running VariantFormer analysis with VCF integration...
⏳ Processing individual genotype data...
Initializing Variant Processor...
Loading gene annotations...
Loading CRE annotations...
Loading variants...
Loaded 1 variants
Loaded 1 variants for processing
Mapped 1 gene-variant pairs
Loading BPE encoder...
Loaded BPE vocabulary from /work/vocabs/bpe_vocabulary_500_using_huggingface.json
Loading model...
Loading Seq2Reg model...
Loading Seq2Reg gene model...
Creating Seq2Gene model...
Model class: <class 'seq2gene.model_combined_modulator.Seq2GenePredictorCombinedModulator'>
Model architecture:
Model: Seq2GenePredictorCombinedModulator
  start_tkn: 96,768 params
  cre_tokenizer: 31,826,153 params
  gene_tokenizer: 31,826,153 params
  gene_map: 787,968 params
  cre_map: 787,968 params
  combined_modulator: 1,157,298,176 params
  tissue_heads: 4,726,273 params
  gene_loss: 0 params
Total number of parameters: 1,227,349,459
Loading checkpoint from /work/_artifacts/v4_ag_epoch9_checkpoint

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Model loaded successfully on cuda
Initializing trainer...
Initialization complete!


Predicting: |          | 0/? [00:00<?, ?it/s]

2025-10-31 18:56:28 - utils.assets - INFO - Downloading from S3: s3://czi-variantformer/model/common/cres_all_genes_manifest.parquet
2025-10-31 18:56:28 - utils.assets - INFO - Loading parquet file: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/youssef/src/variantformer/_artifacts/cres_seqs_manifest.parquet
2025-10-31 18:56:28 - utils.assets - INFO - Validated schema - found columns: {'file_path', 'chromosome', '__index_level_0__', 'population'}
2025-10-31 18:56:28 - utils.assets - INFO - Downloading from S3: s3://czi-variantformer/model/common/reference_genomes/data_split/hg38/cres/data_split/hg38_chr13.pkl.gz
2025-10-31 18:56:29 - utils.assets - INFO - Loading parquet file: /tmp/tmpxv1ndvtt/model/common/cres_all_genes_manifest.parquet
2025-10-31 18:56:29 - utils.assets - INFO - Validated schema - found columns: {'gene_id', 'file_path'}
2025-10-31 18:56:29 - utils.assets - INFO - Downloading from S3: s3://czi-variantformer/model/common/cres_all_genes/ENSG00000185989.10/gene_vocab.csv
20

Predictions saved to /tmp/vep_output_vcf/vep_D2C.parquet
Cleanup complete
✅ VCF-based predictions completed!


In [22]:
# Format scores for VCF-based predictions
vcf_formatted_scores = vep.format_scores(vcf_predictions)
vcf_formatted_scores.head()

gt-exp,variant_id,genes,tissues,chr,pos,ref,alt,variant_type,REF_HG38-0-exp,REF_HG38-1-exp,REF_HG38-2-exp,SAMPLE-0-exp,SAMPLE-1-exp,SAMPLE-2-exp
0,chr13_113978728_A_G,ENSG00000185989.10,whole blood,chr13,113978728,A,G,Gene overlap only,3.182971,3.182971,3.182971,3.242956,3.242956,3.242956


In [23]:
print('REF_HG38-0-exp: Predicted expression for the reference allele in the HG38 reference genome context')
print('REF_HG38-1-exp: Predicted expression for the heterozygous alternate in the HG38 reference genome context')
print('REF_HG38-2-exp: Predicted expression for the homozygous alternative allele in the HG38 reference genome context')
print('SAMPLE-0-exp: Predicted expression for the individual without perturbation at that locus')
print('SAMPLE-1-exp: Predicted expression for the individual with heterozygous alternate at that locus')
print('SAMPLE-2-exp: Predicted expression for the individual with homozygous alternate at that locus')

REF_HG38-0-exp: Predicted expression for the reference allele in the HG38 reference genome context
REF_HG38-1-exp: Predicted expression for the heterozygous alternate in the HG38 reference genome context
REF_HG38-2-exp: Predicted expression for the homozygous alternative allele in the HG38 reference genome context
SAMPLE-0-exp: Predicted expression for the individual without perturbation at that locus
SAMPLE-1-exp: Predicted expression for the individual with heterozygous alternate at that locus
SAMPLE-2-exp: Predicted expression for the individual with homozygous alternate at that locus


In [24]:
# Apply eQTL scoring to VCF results
vcf_final_results = vep.eqtl_scores(vcf_formatted_scores)

print("✅ VCF-based eQTL scoring completed!")

vcf_final_results.head()

✅ VCF-based eQTL scoring completed!


gt-exp,variant_id,genes,tissues,ref,alt,chr,pos,D2C-REF_HG38-2-exp-log2fc,D2C-SAMPLE-2-exp-log2fc
0,chr13_113978728_A_G,ENSG00000185989.10,whole blood,A,G,chr13,113978728,0.0,0.026935


In [25]:
print('D2C-REF_HG38-2-exp-log2fc: Log2 fold change in expression for the homozygous alternative allele in the HG38 reference genome compared to reference homozygous ref')
print('D2C-SAMPLE-2-exp-log2fc: Log2 fold change in expression for the homozygous alternative allele in the individual sample compared to reference')


D2C-REF_HG38-2-exp-log2fc: Log2 fold change in expression for the homozygous alternative allele in the HG38 reference genome compared to reference homozygous ref
D2C-SAMPLE-2-exp-log2fc: Log2 fold change in expression for the homozygous alternative allele in the individual sample compared to reference
